In [9]:
import snowflake.connector
import random
import json
import os
from decimal import Decimal
from dotenv import load_dotenv
load_dotenv()

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse='COMPUTE_WH',
    database='GLOBAL_WATER_DATA_DB2',
    schema='WATER_CONSUMPTION_ANALYSIS2'
)

cursor = conn.cursor()

def fetch_existing_water_data():
    """Fetch existing data from WATER_CONSUMPTION table."""
    cursor.execute("SELECT * FROM WATER_CONSUMPTION")
    existing_data = cursor.fetchall()
    return existing_data

def generate_decimal(min_val, max_val, precision=2):
    """Generate a decimal number with the specified precision."""
    return round(random.uniform(min_val, max_val), precision)

def generate_water_data(num_records=100):
    records = []
    countries = ["USA", "India", "China", "Brazil", "Germany", "Australia", "Canada", "UK", "France", "Japan"]
    scarcity_levels = ["Low", "Moderate", "High", "Severe"]
    
    for _ in range(num_records):
        record = {
            'COUNTRY': random.choice(countries),
            'YEAR': random.randint(2000, 2025),
            'total water consumption (billion cubic meters)': generate_decimal(50, 1000),
            'per capita water use (liters per day)': generate_decimal(50, 500),
            'WATER_SCARCITY_LEVEL': random.choice(scarcity_levels),
            'agricultural water use (%)': generate_decimal(30, 80),
            'industrial water use (%)': generate_decimal(10, 50),
            'household water use (%)': generate_decimal(5, 30),
            'rainfall impact (annual precipitation in mm)': generate_decimal(100, 3000),
            'groundwater depletion rate (%)': generate_decimal(0, 15)
        }
        records.append(record)
    
    return records

try:
    existing_data = fetch_existing_water_data()
    print(f"Fetched {len(existing_data)} existing records from WATER_CONSUMPTION")
    
    new_records = generate_water_data(100)
    json_file = "water_data.json"
    
    with open(json_file, "w") as f:
        for record in new_records:
            json.dump(record, f)
            f.write("\n")
    
    cursor.execute("CREATE TEMPORARY STAGE IF NOT EXISTS temp_stage")
    cursor.execute(f"PUT file://{json_file} @temp_stage")
    
    cursor.execute("""
    COPY INTO WATER_CONSUMPTION
    FROM @temp_stage/water_data.json.gz
    FILE_FORMAT = (TYPE = 'JSON')
    MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    """)
    
    print(f"Successfully loaded {len(new_records)} new records into WATER_CONSUMPTION")
except Exception as e:
    print(f"Error loading data: {e}")
finally:
    try:
        cursor.execute("DROP STAGE IF EXISTS temp_stage")
        os.remove(json_file)
    except Exception as e:
        print(f"Error during cleanup: {e}")
    cursor.close()
    conn.close()
    print("Connections closed")


Fetched 5100 existing records from WATER_CONSUMPTION
Successfully loaded 100 new records into WATER_CONSUMPTION
Connections closed
